# Introduction à Spark SQL

Spark SQL est un module d'Apache Spark, qui facilite la mise en place de traitement sur des données à haute volumétrie :
 * **structurées** : la donnée est stockée sous un format standardisé (CSV, JSON, Avro, Parquet...) et répond à une structure partagée (ie. schéma) répondant à un besoin technique ou métier
 * **semi-structurées** : la donnée est stockée sous un format standardisé, mais sa structure interne n'est pas connue par avance.

Spark SQL offre une interface pour interagir avec les données via le langage SQL, ainsi que des fonctionnalités pour la lecture et l'écriture de données dans divers formats. Spark SQL facilite l'intégration entre le traitement des données relationnelles et le traitement distribué à grande échelle en utilisant les DataFrames et les Datasets, deux structures de données immuables.

## Préambule

In [1]:
import $ivy.`org.apache.spark::spark-core:3.3.2`
import $ivy.`org.apache.spark::spark-sql:3.3.2`

import $ivy.$                                   

import $ivy.$                                  

In [2]:
// Avoid disturbing logs
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.rdd._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    // L'appel ci-dessous sert à donner un nom à votre application
    // Ce apparaîtra notamment dans la Spark UI
    .appName("Sales Analysis - SparkSQL")
    .getOrCreate()
}

import spark.implicits._

// Ce script fournit que élément supplémentaires pour rendre l'affichage plus confortable
import $file.^.internal.spark_helper, spark_helper._

Compiling /home/jovyan/work/internal/spark_helper.scLoading spark-stubs
Getting spark JARs


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark UI

import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._

import org.apache.spark.sql.functions._

import org.apache.spark.rdd._


spark: SparkSession = org.apache.spark.sql.SparkSession@33adb981
import spark.implicits._

// Ce script fournit que élément supplémentaires pour rendre l'affichage plus confortable

import $file.$                      , spark_helper._

## Lecture d'un fichier avec Spark SQL

Nous allons récupérer le fichier `orders.csv` et réaliser des analyses sur ce fichier.

Commençons par afficher un extrait de son contenu.

In [3]:
shell("cat orders.csv", limit=10)

/home/jovyan/work/02_spark_sql $ cat orders.csv 
id,client,timestamp,product,price
87365481,XztHU0aeUckvR7AC,2022-11-14T13:25:36,café allongé,1.4
42761208,t_CUBr6tyTQxGj2X,2022-11-14T13:29:46,café crème,2.5
90524048,hdVMQjoIgOov09zb,2022-11-14T13:34:09,chocolat chaud,2.6
09935741,hdVMQjoIgOov09zb,2022-11-14T13:37:10,chocolat chaud,2.6
03486136,TX7wC0pTqCRlCOhi,2022-11-14T13:40:52,expresso,1.1
46727424,H-Mp22FLe99MNhRa,2022-11-14T13:45:13,décaféiné,1.4
97190478,oplTx8h-38G3be4c,2022-11-14T13:50:05,décaféiné,1.4
49642764,TX7wC0pTqCRlCOhi,2022-11-14T13:53:27,expresso,1.1
33866371,JBoCs7rWb_jEs87W,2022-11-14T13:56:58,double café,2.6

### Lecture : première approche
La récupération du contenu d'un fichier avec Spark SQL va s'avérer beaucoup plus simple qu'avec Spark Core, car Spark SQL est fourni avec un ensemble de codec pour gérer les formats CSV, JSON, texte, binaire, Avro, Parquet, ORC.

In [7]:
val dataframe: DataFrame =
  spark.read
    // indique que le fichier contient une ligne d'en-tête qui servira
    // pour nommer les champs
    .option("header", true)
    // demande à Spark SQL de tenter de déterminer le type des colonnes
    .option("inferSchema", true)
    // lecture du fichier au format CSV
    .csv("orders.csv")

dataframe.showHTML(limit=10,truncate=40)

csv at cmd6.sc:9

1 / 1

csv at cmd6.sc:9

1 / 1

showHTML

1 / 1

id,client,timestamp,product,price
87365481,XztHU0aeUckvR7AC,2022-11-14 13:25:36.0,café allongé,1.4
42761208,t_CUBr6tyTQxGj2X,2022-11-14 13:29:46.0,café crème,2.5
90524048,hdVMQjoIgOov09zb,2022-11-14 13:34:09.0,chocolat chaud,2.6
9935741,hdVMQjoIgOov09zb,2022-11-14 13:37:10.0,chocolat chaud,2.6
3486136,TX7wC0pTqCRlCOhi,2022-11-14 13:40:52.0,expresso,1.1
46727424,H-Mp22FLe99MNhRa,2022-11-14 13:45:13.0,décaféiné,1.4
97190478,oplTx8h-38G3be4c,2022-11-14 13:50:05.0,décaféiné,1.4
49642764,TX7wC0pTqCRlCOhi,2022-11-14 13:53:27.0,expresso,1.1
33866371,JBoCs7rWb_jEs87W,2022-11-14 13:56:58.0,double café,2.6
55962364,t_CUBr6tyTQxGj2X,2022-11-14 16:00:30.0,expresso,1.1


dataframe: DataFrame = [id: int, client: string ... 3 more fields]

**Ce qu'il faut voir**

Dans la Spark UI, vous pouvez voir un nouvel onglet dans la barre du haut intitulé "SQL / DataFrame". En cliquant dessus, vous verrez apparaître les requêtes exécutées par Spark SQL. Si vous cliquez sur une requête, vous verrez un diagramme représentant le plan d'exécution et dans la partie "Details" une représentation textuelle du plan d'exécution.

Affichons le schéma de notre dataframe.

In [5]:
dataframe.printSchema

root
 |-- id: integer (nullable = true)
 |-- client: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- product: string (nullable = true)
 |-- price: double (nullable = true)



**Ce qu'il faut voir**

Avec l'instruction `dataframe.printSchema`, nous pouvons voir que Spark a réussi à déterminer le schéma des données du fichier. Ce qui inclut le fait de déterminer le nom des colonnes et de déterminer le type des colonnes (grâce à l'option `inferSchema` pour ce dernier). Cependant, l'option `inferSchema` a deux problèmes majeurs :

 * Il nécessite une lecture supplémentaire du fichier (sur un extrait). Si vous regardez dans le Spark UI, vous verrez deux étapes de lecture CSV.
 * Il peut se tromper.

### Lecture : deuxième approche
Nous allons maintenant relire le fichier CSV, mais cette fois en fournissant directement un schéma.

In [3]:
val dataframe: DataFrame =
  spark.read
    // indique que le fichier contient une ligne d'en-tête qui servira
    // pour nommer les champs
    .option("header", true)
    // force le schéma
    .schema("id STRING, client STRING, timestamp TIMESTAMP, product STRING, price DOUBLE")
    // lecture du fichier au format CSV
    .csv("orders.csv")

dataframe.showHTML(limit=10,truncate=40)

showHTML

1 / 1

id,client,timestamp,product,price
87365481,XztHU0aeUckvR7AC,2022-11-14 13:25:36.0,café allongé,1.4
42761208,t_CUBr6tyTQxGj2X,2022-11-14 13:29:46.0,café crème,2.5
90524048,hdVMQjoIgOov09zb,2022-11-14 13:34:09.0,chocolat chaud,2.6
09935741,hdVMQjoIgOov09zb,2022-11-14 13:37:10.0,chocolat chaud,2.6
03486136,TX7wC0pTqCRlCOhi,2022-11-14 13:40:52.0,expresso,1.1
46727424,H-Mp22FLe99MNhRa,2022-11-14 13:45:13.0,décaféiné,1.4
97190478,oplTx8h-38G3be4c,2022-11-14 13:50:05.0,décaféiné,1.4
49642764,TX7wC0pTqCRlCOhi,2022-11-14 13:53:27.0,expresso,1.1
33866371,JBoCs7rWb_jEs87W,2022-11-14 13:56:58.0,double café,2.6
55962364,t_CUBr6tyTQxGj2X,2022-11-14 16:00:30.0,expresso,1.1


dataframe: DataFrame = [id: string, client: string ... 3 more fields]

**Ce qu'il faut voir**

Le fait de fournir un schéma va inciter Spark SQL à ne pas réaliser des analyses préalables ou des vérifications. Nous voyons, en effet, que l'ensemble du process est réduit à un job au lieu de trois.

In [4]:
import java.sql.Timestamp

case class Order(
  id:        String,
  clientId:  String,
  timestamp: Timestamp,
  product:   String,
  price:     Double
)

val orders: Dataset[Order] =
  dataframe
    .withColumnRenamed("client", "clientId")
    .as[Order]

orders.showHTML(limit=10, truncate=40)

showHTML

1 / 1

id,clientId,timestamp,product,price
87365481,XztHU0aeUckvR7AC,2022-11-14 13:25:36.0,café allongé,1.4
42761208,t_CUBr6tyTQxGj2X,2022-11-14 13:29:46.0,café crème,2.5
90524048,hdVMQjoIgOov09zb,2022-11-14 13:34:09.0,chocolat chaud,2.6
09935741,hdVMQjoIgOov09zb,2022-11-14 13:37:10.0,chocolat chaud,2.6
03486136,TX7wC0pTqCRlCOhi,2022-11-14 13:40:52.0,expresso,1.1
46727424,H-Mp22FLe99MNhRa,2022-11-14 13:45:13.0,décaféiné,1.4
97190478,oplTx8h-38G3be4c,2022-11-14 13:50:05.0,décaféiné,1.4
49642764,TX7wC0pTqCRlCOhi,2022-11-14 13:53:27.0,expresso,1.1
33866371,JBoCs7rWb_jEs87W,2022-11-14 13:56:58.0,double café,2.6
55962364,t_CUBr6tyTQxGj2X,2022-11-14 16:00:30.0,expresso,1.1


import java.sql.Timestamp


defined class Order
orders: Dataset[Order] = [id: string, clientId: string ... 3 more fields]

## Trouvez le produit le plus vendu (ID du produit et quantité totale vendue)

### Première approche : utilisation de l'API Spark SQL

In [5]:
orders
  .groupBy($"product")
  .agg(count(lit(1)).as("count"))
  .orderBy($"count".desc)
  .showHTML()

showHTML

1 / 1

showHTML

1 / 1

product,count
expresso,3215
café,1804
café allongé,1301
décaféiné,1004
noisette,798
café crème,728
chocolat chaud,604
double café,546


### Deuxième approche : utilisation d'une requête SQL

In [6]:
orders.createTempView("orders")
spark.sql("""
SELECT product, count(1) as count
FROM orders
GROUP BY product
ORDER BY count DESC
""").showHTML()

showHTML

1 / 1

showHTML

1 / 1

product,count
expresso,3215
café,1804
café allongé,1301
décaféiné,1004
noisette,798
café crème,728
chocolat chaud,604
double café,546


**Ce qu'il faut voir**

Si vous regardez le plan d'exécution de cette requête et que vous le comparez au plan d'exécution obtenu à travers l'utilisation de l'API Spark SQL, vous remarquerez que ces deux plans d'exécution sont identiques. Ce qui indique bien que les deux approches font exactement la même chose et qu'elles le font avec les mêmes performances.

Ainsi, Spark SQL vous donne la possibilité d'utiliser le langage qui vous convient le plus, tout en ayant le même comportement de la part de Spark. Ceci est vrai dans la majorité des cas, si vous vous tenez aux fonctions de base fournies par Spark SQL. C'est moins vrai dès que vous introduisez des éléments personnalisés (eg. UDF).